In [2]:
! pip install -r requirements.txt --quiet

In [1]:

from os import environ
from dotenv import load_dotenv
import jsonref
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import OpenApiTool, OpenApiAnonymousAuthDetails
import requests

load_dotenv(override=True)

True

## Create a project client

In [2]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=environ["AZURE_AI_PROJECT_CONNECTION_STRING"],
)

## Create the OpenAPI Spec tool definition

In [3]:
spec_url = environ['OPENAPI_URL']
response = requests.get(spec_url)
response.raise_for_status()


openapi_spec = response.json()

auth = OpenApiAnonymousAuthDetails()

openapi = OpenApiTool(name="get_energy_usage", spec=openapi_spec, description="Retrieve data center energy, power, or consumption topic", auth=auth)


## Create an agent and a thread

In [4]:
instructions = """
You are a helpful and knowledgeable assistant that helps users understand and monitor data center energy usage.

Use the provided tools to retrieve accurate, real-time, and historical data on energy consumption, efficiency metrics (such as PUE), carbon footprint, and resource utilization across different data center regions or racks.

Always respond with concise and clear summaries, including units of measurement (e.g., kWh, %). If applicable, compare usage across regions or time ranges to highlight trends or inefficiencies.

If the user asks something unrelated to data center energy or the information is not available via the provided tools, respond with: "I'm unable to find that information based on the current data."

Only use the tools provided to retrieve answers.
"""



agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="Data Center Energy Usage Agent",
    instructions=instructions,
    tools=openapi.definitions
)

print(f"Created agent, ID: {agent.id}")

# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")



Created agent, ID: asst_qvaQ6dZu2bfRiJYAD0m66uWI
Created thread, ID: thread_7ReLSOjQPl1VVhNndpdPptOq


## Create a run and check the output

In [5]:
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="What data centers have the highest co2 emissions?",
)
print(f"Created message, ID: {message.id}")

# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")

Created message, ID: msg_hKsPGcuDUuThHMfuCwdaPQWj
Run finished with status: RunStatus.COMPLETED
Messages: {'object': 'list', 'data': [{'id': 'msg_om6rapTCOscpl7PGkoTWsxF3', 'object': 'thread.message', 'created_at': 1744910542, 'assistant_id': 'asst_qvaQ6dZu2bfRiJYAD0m66uWI', 'thread_id': 'thread_7ReLSOjQPl1VVhNndpdPptOq', 'run_id': 'run_EDWhFLPUndRTyIJwV9KFfLXn', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'Based on recent data, the data centers with the highest CO2 emissions are:\n\n1. **DC-CHI2 (Zone C3)** - 153.31 kg of CO2 emissions recently recorded.\n2. **DC-NYC1 (Zone A1)** - 125.15 kg of CO2 emissions recently recorded.\n3. **DC-CHI2 (Zone B2)** - 137.42 kg of CO2 emissions recently recorded.\n4. **DC-NYC1 (Zone C3)** - 126.48 kg of CO2 emissions recently recorded.\n\nThese figures reflect high-emission zones due to factors like energy source reliance (grid/diesel/battery) and operational inefficiencies.', 'annotations': []}}], 'attachments': [], 'metada

## Delete the assistant when done

Comment the code below to save the agent to your Azure AI Froundry project.


In [ ]:

project_client.agents.delete_agent(agent.id)
print("Deleted agent")